# 음성 분류 경진대회 첫 번째 베이스라인

이번 대회는 음성 데이터를 이용하여 0~9까지 숫자를 분류하는 대회입니다.

처음 음성 데이터를 접하시는 분들에게는 어디부터 어떻게 시작해야 하는지 막막하실 겁니다.

### 누구나 할 수 있는 데이터 분석
딥러닝, 머신러닝, 모델, 학습, 회귀분석 .... 다 몰라도 괜찮습니다!

누구나 할 수 있는 방식으로 간단하게 데이터 분석 & 예측을 진행해보겠습니다.


# 분석 환경 준비

* 오디오 전처리를 위한 라이브러리

In [1]:
import librosa
import librosa.display as dsp
from IPython.display import Audio

* 데이터 전처리를 위한 라이브러리

In [2]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

* 모델의 재현성을 위하여 random seed 고정

In [3]:
import random

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(929)

# 데이터 불러오기
분석하려는 데이터를 가져오는 작업

먼저 csv 파일을 불러와서 label이 어떻게 되어있는지 살펴보겠습니다.

In [4]:
import pandas as pd
train = pd.read_csv('data/train.csv')
train.head()

,file_name,label
0,001.wav,9
1,002.wav,0
2,004.wav,1
3,005.wav,8
4,006.wav,0


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  400 non-null    object
 1   label      400 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 6.4+ KB


label이 int 타입으로 되어있는 것을 확인할 수 있습니다.

# 데이터 전처리

## 음성 데이터 Load

그럼 이제 음성데이터를 한번 살펴봅시다.

소리는 기본적으로 특정 주파수를 가지는 sin함수들의 합입니다.

특정 시간에 주파수 성분이 어떻게 구성되어 있는지 확인할 수 있는데요. 

음성 데이터 분석을 할 때 주파수 분석 기법을 많이 사용합니다. (파형 자체를 이용하기도 합니다!) 

주파수 분석은 크게 3단계로 이루어지는데, 이번 포스팅에서는 그 중에서도


음성 데이터를 학습하기 위해서는 아날로그 데이터로 되어있는 음성 데이터를 디지털 신호로 변환해야 합니다. 

따라서 librosa 라이브러리를 사용해 음성 데이터를 load 를 해봅시다.

우선 예시로 train 음성 데이터의 첫번째 데이터를 load 해보겠습니다.

In [6]:
data, sample_rate = librosa.load('data/train/001.wav', sr = 16000)
print('sample_rate:', sample_rate, ', audio shape:', data.shape)
print('length:', data.shape[0]/float(sample_rate), 'secs')

sample_rate: 16000 , audio shape: (10192,)
length: 0.637 secs


sampling rate의 의미는 초당 16000개(16000Hz 주파수)의 샘플을 가지고 있는 데이터라는 의미입니다. (1초에 음성 신호를 16000번 sampling) 

sampling rate의 defult값은 22050Hz인데, 16000Hz으로 설정한 이유는 사람의 목소리는 대부분 16000Hz 안에 포함된다고 합니다.

또한 audio shape와 sampling rate를 이용해서 오디오 길이 계산을 할 수 있습니다.

계산 결과, 길이는 0.637 secs 가 되는군요!

그럼 디지털 신호와 라벨을 포함한 train_wav 데이터프레임을 생성하겠습니다.

In [7]:
def train_dataset():
    folder = "data/train/"
    dataset = []
    for file in tqdm(os.listdir(folder),colour='green'):
        if 'wav' in file:
            abs_file_path = os.path.join(folder,file)
            data, sr = librosa.load(abs_file_path, sr = 16000)
            class_label = int(train[train.file_name == file].label)
            dataset.append([data,class_label])
    
    print("Dataset 생성 완료")
    return pd.DataFrame(dataset,columns=['data','label'])

In [8]:
train_wav = train_dataset()

100%|██████████| 401/401 [00:12<00:00, 31.46it/s]

Dataset 생성 완료


In [9]:
train_wav.head()

,data,label
0,"[0.00029359042, 0.0004896918, 0.0004225315, 0....",0
1,"[0.00021014328, 0.0003359131, 0.00028251947, 0...",1
2,"[-5.409059e-05, -0.00013994443, -0.00013878023...",2
3,"[-1.7762668e-05, -5.3217824e-05, -1.977646e-05...",4
4,"[-9.07404e-05, -0.00016544455, -0.00015305405,...",6


## 음성 데이터 특징 추출

음성 데이터를 load 했으면 이 음성 데이터의 특징을 추출해야 합니다.

음성 raw data를 그대로 사용하면 파라미터가 너무 많아지기도 하고 데이터 용량이 너무 커집니다.

따라서 입력된 신호에서 노이즈 및 배경 소리로 부터 실제로 유용한 소리의 특징을 추출하는 것이죠.


### 퓨리에 변환

음성 데이터를 분석하기 위하여 주파수(frequency)를 성분을 뽑아내야 한다면 퓨리에 변환(Fourier Transform)을 해야 합니다.

퓨리에 변환은 간단히 설명하자면 **'입력 신호를 다양한 주파수를 가지는 주기함수들로 분해하는 것'** 입니다.

주기함수들을 분해함으로써 음성 데이터에서 노이즈 및 배경 소리로 부터 실제로 유용한 소리의 데이터, 즉 특징을 추출하는 것 입니다.



### MFCC (Mel-frequency cepstral coefficients)


하지만 음성데이터 전체를 퓨리에 변환을 한다면,

예를들어 "안녕하세요"라고 하더라도, 어떤 사람은 1초, 어떤 사람은 3초가 걸릴 수도 있습니다.

따라서 이 천차만별인 길이에 대하여 같은 "안녕하세요"라는 음성이라고 학습시키기는 어려울 것입니다.

위와 같은 문제를 해결하기 위하여 MFCC (Mel-frequency cepstral coefficients) 알고리즘을 이용합니다.

MFCC는 음성데이터를 특징벡터화 해주는 알고리즘입니다.

입력된 소리 전체를 대상으로 하는 것이 아니라, 

사람이 인지하기 좋은 **Mel-scale로 음성데이터를 모두 20~40ms로 나누어** 이 구간에 대한 스펙트럼을 분석하여 

**퓨리에 변환**을 한 특징 추출 방법입니다.

사람의 음성은 20~40ms 사이에서는 음소(현재 내고 있는 발음)가 바뀔 수 없다는 연구결과들을 기반으로 

음소는 해당 시간내에 바뀔 수 없다고 가정합니다.

따라서 MFCC에서는 음성데이터를 모두 20~40ms 단위로 쪼개고, 

쪼갠 단위에 대해서 Mel 값을 뽑아서 Feature로 사용합니다.

파이썬에서는 librosa.feature.mfcc(wav) 메소드 이용하여 

손쉽게 MFCC 알고리즘을 이용합니다.


#### **Mel-scale**

**Mel은 사람의 달팽이관을 모티브로 따온 값**입니다.

달팽이관은 주파수가 낮은 대역에서는 변화하는 주파수를 잘 감지하는데, 

주파수가 높은 대역에서는 주파수 감지를 잘 하지 못합니다. (주파수 간격이 넓어짐)

이러한 원리를 이용해서 filter, scaling 해줄 수 있는데, 이때 이 기준을 Mel-Scale 이라고 합니다.


#### Argument 

* y : audio data
   
*  sr : sampling rate
   
* n_mfcc :  return 될 mfcc의 개수를 정해주는 파라미터. 더 다양한 데이터 특징을 추출하기 위해서 증가 시킵니다.
   
* n_fft : frame의 length를 결정하는 파라미터. n_fft를 설정하면 window size가 디폴트 값으로 n_fft가 됩니다.        

    사람의 목소리는 대부분 16000Hz 안에 포함이 되는데, 일반적으로 자연어 처리에서는 음성을 25m의 크기를 기본으로 하고 있습니다.        
    
    (ex. 16000Hz인 음성에서는 25m의 음성의 크기를 가지고 있으면 n_fft는 16000 * 0.025 = 400 (sampling rate * frame_length = n_fft)가 됩니다.)
        
* hop_length : 윈도우 길이를 나타냅니다. 길이만큼 옆으로 가면서 데이터를 읽습니다.

    hop_length도 마찬가지로 window 간의 거리이므로 sampling rate * frame_stride 가 됩니다.

In [10]:
def extract_features(file):
    audio, sample_rate = librosa.load(file, sr = 16000)
    extracted_features = librosa.feature.mfcc(y=audio,
                                              sr=sample_rate,
                                              n_mfcc=40)

    extracted_features = np.mean(extracted_features.T,axis=0)
    return extracted_features

In [11]:
extract_features('data/train/001.wav') 

array([-5.4309741e+02,  1.0221236e+02, -1.0131239e+01,  4.5035259e+01,
        7.5383821e+00,  1.1087791e+01, -1.2066618e+01, -6.1290884e+00,
       -2.0615113e+00,  4.3403912e+00, -7.9769449e+00, -2.3532255e+00,
       -9.4535418e+00,  1.3950575e+01, -1.2931410e+01, -3.1012371e+00,
       -3.1802003e+00, -2.2292783e+00, -6.0094471e+00, -5.1039085e+00,
       -1.4015083e+01,  3.1813147e+00, -6.9873381e+00,  5.3114867e-01,
       -4.3120513e+00,  1.0081086e+00, -1.4859858e+00,  4.2523170e+00,
        2.5631633e+00,  2.8835654e+00,  2.3709497e+00,  1.5794464e+00,
       -3.6886792e+00, -4.3605208e-02, -1.4032007e+00, -4.1899872e+00,
       -5.2867079e+00, -8.9757144e-01, -3.1176820e+00, -3.4578696e-01],
      dtype=float32)

그럼 train 셋 데이터 전체를 MFCC 알고리즘을 이용하여 특징을 추출하겠습니다.

In [12]:
def preprocess_train_dataset(data):
    mfccs = []
    for i in data:
        extracted_features = librosa.feature.mfcc(y=i,
                                              sr=16000,
                                              n_mfcc=40)
        extracted_features = np.mean(extracted_features.T,axis=0)
        mfccs.append(extracted_features)
            
    return mfccs

mfccs = preprocess_train_dataset(train_wav.data)
mfccs = np.array(mfccs)

In [13]:
mfccs

array([[-6.03562805e+02,  1.00078056e+02,  3.29141903e+00, ...,
         4.53813457e+00, -1.95242643e-01,  7.79115379e-01],
       [-6.49387817e+02,  1.50144684e+02,  1.44454021e+01, ...,
         3.35813475e+00,  7.45637059e-01,  1.56931162e+00],
       [-6.09140564e+02,  1.14499817e+02,  2.11241226e+01, ...,
         6.78267717e-01, -1.52308869e+00,  3.69100571e-01],
       ...,
       [-5.89701050e+02,  1.23524132e+02, -7.10499859e+00, ...,
         7.91468954e+00,  3.60656190e+00,  3.15482616e+00],
       [-5.69479797e+02,  7.28337784e+01, -9.92106438e+00, ...,
         1.79753666e+01,  1.67299080e+01,  1.12663355e+01],
       [-5.67246521e+02,  1.89183009e+00,  5.29523735e+01, ...,
         6.42307711e+00,  1.74153101e+00,  6.90778399e+00]], dtype=float32)

## 변수 및 모델 정의

이제 본격적으로 모델을 설계하기 위해 데이터를 독립변수(X)와 종속변수(y) 로 나누어 줍니다.

우리의 목적은 주어진 음성 특징 데이터를 이용하여 라벨을 분류하는 것입니다.

그럼 음성 특징 데이터가 X, 라벨이 y가 되겠죠?

또한 모델을 만들기 앞서 내가 만들 모델이 유의미한지 알아야 좋은 결론을 도출 할 수 있겠죠?   

<br> 

하지만 단순히 주어진 데이터로 train셋 데이터를 학습시켜서 목표값인 test 셋에 집어 넣어 예측값을 도출하면,   
모델이 적합한지 알 방법이 없습니다.   

<br> 

따라서 train 셋에서 60%를 학습을 시키는데 쓰고, 나머지 40%를 모델이 유의미한지 검증하는데 사용할 것입니다.   
이후 유의미하다면, 실제로 제출할 모델을 만들 때, train 셋 100%를 학습을 시키면 되겠죠?

<br>

자 그럼 train 셋에서 학습시킬 셋 train_data와 검증할 셋 val_data를 만들어 봅시다!

In [14]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(mfccs, train_wav.label, test_size=0.4)

In [15]:
print('학습시킬 train 셋 : ', train_X.shape, train_y.shape)
print('검증할 val 셋 : ', test_X.shape, test_y.shape)

학습시킬 train 셋 :  (240, 40) (240,)
검증할 val 셋 :  (160, 40) (160,)


위의 데이터 모양을 보면, 240개의 데이터를 학습시켜 160개의 데이터로 성능을 검증시킬 것 입니다!

## 랜덤 포레스트 모델 학습

대표적인 ML모델의 하나인 RandomForestClassifier 모델을 사용해봅시다.

Classifier 모델을 사용하는 이유는, 우리가 이번 대회에서 예측해야할 0~9 라벨이 분류 변수이기 때문입니다.

RandomForest 알고리즘에서 사용되는 의사 결정 트리는 feature 별 가지치기를 통해 데이터를 학습하는 알고리즘입니다.

의사 결정 나무에 대한 자세한 개념 설명은 데이콘 [오늘의 파이썬](https://dacon.io/competitions/open/235698/talkboard/403509?page=1&dtype=recent)을 참고해 주세요!

의사 결정 나무는 데이터를 쉽게 학습하는 알고리즘이지만 하나의 의사 결정 나무ㅌ를 사용하는 것은 과적합(overfitting)의 문제를 발생 시킬 수 있습니다.

여기서 과적합(overfitting)이란 학습 데이터에 과하게 학습되어 새로운 데이터를 예측하지 못하는 문제를 말합니다.

쉽게 말해 우리가 공부를 할때 암기식으로 공부를 하여 새로운 문제를 해결하지 못하는 것 입니다!

RandomForest는 여러개의 의사 결정 나무를 활용하여 과적합의 문제를 해결합니다.

여러개의 의사 결정 트리를 활용하면 하나의 트리에 대한 의존도가 낮아집니다.

이는 하나의 트리가 과적합이 되어도 전체 모델은 과적합의 문제에서 벗어날 수 있음을 의미합니다.

sklearn 패키지를 이용하면 직접 RandomForest를 구현하지 않고 모델을 사용할 수 있습니다.

이번 베이스라인에서는 sklearn에서 제공하는 RandomForestClassifier 활용해보겠습니다.

In [16]:
from sklearn.ensemble import RandomForestClassifier

# 모델 선언
model = RandomForestClassifier()

# 모델 학습
model.fit(train_X, train_y)

RandomForestClassifier()

## 모델 성능 확인

굉장히 간단하게 학습이 끝났습니다.

이제 이 모델이 실제로 데이터를 얼마나 잘 맞추는지 확인해볼까요?

In [17]:
# 먼저 점수를 메기는 방법인 평가 지표(Metric)를 정의합니다.
import numpy as np

def ACCURACY(true, pred):   
    score = np.mean(true==pred)
    return score

In [18]:
# 모델의 예측과 실제 정답값을 비교합니다.
prediction = model.predict(test_X)

score = ACCURACY(test_y, prediction)

print(f"모델의 정확도는 {score*100:.2f}% 입니다")

모델의 정확도는 63.75% 입니다


## test.csv 분류하기

이제 본격적으로 대회에서 주어진 정답이 없는 test 데이터의 라벨을 예측해보겠습니다.

전처리 모델 학습 단계에서 test 데이터를 사용하면 data leakage로 간주되니

test.csv는 반드시 모델 학습이 끝난 뒤 예측시에만 등장해야 함에 주의해주세요!

train.csv를 이용한 전처리/학습을 test.cvs에 적용하는 것은 허용되지만, test.csv를 전처리/모델링에 직접적으로 이용하면 실격입니다.

In [19]:
test = pd.read_csv('data/test.csv')
test.head()

,file_name
0,003.wav
1,008.wav
2,010.wav
3,015.wav
4,024.wav


In [20]:
def test_dataset():
    folder = "data/test/"
    dataset = []
    for file in tqdm(os.listdir(folder),colour='green'):
        if 'wav' in file:
            abs_file_path = os.path.join(folder,file)
            data, sr = librosa.load(abs_file_path, sr = 16000)
            dataset.append([data, file])
    
    print("Dataset 생성 완료")
    return pd.DataFrame(dataset,columns=['data', 'file_name'])

In [21]:
test_wav = test_dataset()

100%|██████████| 200/200 [00:05<00:00, 33.76it/s]

Dataset 생성 완료


In [22]:
mfccs_2 = preprocess_train_dataset(test_wav.data)
mfccs_2 = np.array(mfccs_2)

트레인 셋과 검증 셋으로 나누어 성능을 확인해 준 것을 하나의 트레인 셋으로 통합하여

다시 모델을 만들어주겠습니다.

In [23]:
# 모델 선언
model = RandomForestClassifier()

# 모델 학습
model.fit(mfccs, train_wav.label)

# 모델 예측
prediction = model.predict(mfccs_2)

test_wav['label'] = prediction

In [24]:
test_wav.head()

,data,file_name,label
0,"[-3.2772932e-05, -7.182783e-05, -5.9236434e-05...",454.wav,3
1,"[-5.452518e-05, -9.22947e-05, -0.00010464665, ...",566.wav,5
2,"[-0.0003841589, -0.00062216586, -0.00053581083...",173.wav,3
3,"[-0.0003681986, -0.0006039656, -0.0005450903, ...",583.wav,6
4,"[-0.00013396672, -0.0002302016, -0.00017955233...",589.wav,0


In [28]:
testset = test_wav[['file_name', 'label']]

pred_df = testset.copy()
pred_df = pred_df.sort_values(by=[pred_df.columns[0]], ascending=[True]).reset_index(drop=True)
pred_df.head()

,file_name,label
0,003.wav,0
1,008.wav,9
2,010.wav,3
3,015.wav,3
4,024.wav,2


## 제출하기

submission에 예측한 값 preds를 넣어줍시다.

In [29]:
submission = pd.read_csv('data/sample_submission.csv')
submission['label'] = pred_df['label']
submission.head()

,file_name,label
0,003.wav,0
1,008.wav,9
2,010.wav,3
3,015.wav,3
4,024.wav,2


값이 배열안에 정상적으로 잘 들어간 것을 확인할 수 있습니다.

submission을 csv 파일로 저장합니다.   

index=False란 추가적인 id를 부여할 필요가 없다는 뜻입니다.   

정확한 채점을 위해 꼭 index=False를 넣어주세요.

In [30]:
submission.to_csv('data/saved/submit.csv', index=False)

이렇게 생성된 submit.csv 파일을 데이콘 대회 페이지에 업로드 & 제출하여 결과를 확인해보세요!

문제를 해결하기 위한 여러분의 방법을 코드 공유 게시판에 공유해주세요

좋아요와 댓글을 합산하여 가장 높은 점수를 얻으신 분께 데이콘 후드가 제공됩니다!